<a href="https://colab.research.google.com/github/harunlakodla/colab-socket-image-pyngrok/blob/main/colab_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyngrok
!git clone https://github.com/AlexeyAB/darknet
!pip install websockets
!pip install websocket-client
!pip install adafruit-io

In [ ]:
from Adafruit_IO import Client, RequestError, Feed


ADAFRUIT_IO_KEY = ''
ADAFRUIT_IO_USERNAME = ''
aio = Client(ADAFRUIT_IO_USERNAME, ADAFRUIT_IO_KEY)

try:
    server_url = aio.feeds('server-url')
except RequestError: # Doesn't exist, create a new feed
    feed = Feed(name="server-url")
    server_url = aio.create_feed(feed)

try:
    server_port = aio.feeds('server-port')
except RequestError: # Doesn't exist, create a new feed
    feed = Feed(name="server-port")
    server_port = aio.create_feed(feed)


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
#!ls /mydrive

In [ ]:
!mkdir /mydrive/yolov3
!ls

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists
darknet  gdrive  sample_data


In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
!make

In [ ]:
def detect(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline
  
  image = path
  
  plt.imshow(np.real(image))
  plt.show()

In [ ]:
from websocket import create_connection

import base64,cv2,threading,json,asyncio,websockets,random
import numpy as np

def connect(url,port):
    ws = create_connection("ws://"+str(url)+":"+str(port))
    ws.send("Hello, World")
    result = ws.recv()
    ws.close()
    im_bytes = base64.b64decode(result.decode("utf-8"))
    im_arr = np.frombuffer(im_bytes, dtype=np.uint8)  # im_arr is one-dim Numpy array
    img = cv2.imdecode(im_arr, flags=cv2.IMREAD_COLOR)
    return img


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline



def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers


def draw_prediction(img,classes, class_id, confidence, x, y, x_plus_w, y_plus_h):

    print(classes[class_id])
    label = str(classes[class_id])

    aio.send_data(vegetable_and_fruit.key, label)

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [ ]:
#%cd ../

%cd ./content

!wget https://pjreddie.com/media/files/yolov3.weights

!cp ./darknet/cfg/yolov3.cfg .
!cp ./darknet/data/coco.names ./classes.txt
!ls

[Errno 2] No such file or directory: './content'
/content
--2021-07-17 22:00:11--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.2’

yolov3.weights.2    100%[===================>] 236.52M   107MB/s    in 2.2s    

2021-07-17 22:00:13 (107 MB/s) - ‘yolov3.weights.2’ saved [248007048/248007048]

classes.txt  gdrive	  yolov3.cfg	  yolov3.weights.1
darknet      sample_data  yolov3.weights  yolov3.weights.2


In [ ]:
import time

while True:
    # image = cv2.imread('/content/gdrive/My Drive/resim.jpg')

    server_url_data = aio.receive(server_url.key)
    server_port_data = aio.receive(server_port.key)
    try:
      image=connect(str(server_url_data.value),int(server_port_data.value))
    except Exception:
      time.sleep(2)
    # except WebSocketConnectionClosedException:
    #   time.sleep(2)
    #   pass

    Width = image.shape[1]
    Height = image.shape[0]
    scale = 0.00392

    classes = None

    #with open('/mydrive/yolov3/last_train/classes.txt', 'r') as f:
    with open('./classes.txt', 'r') as f:
        classes = [line.strip() for line in f.readlines()]
    COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

    # net = cv2.dnn.readNet('/mydrive/yolov3/last_train/yolov3_custom_last.weights', '/mydrive/yolov3/last_train/yolov3_custom.cfg')
    net = cv2.dnn.readNet('./yolov3.weights', './yolov3.cfg')

    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

    net.setInput(blob)

    outs = net.forward(get_output_layers(net))

    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.5
    nms_threshold = 0.4


    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])


    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        draw_prediction(image,classes, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

    detect(image)